In [1]:
import os
import json
import subprocess
from typing import Dict, List, Any, Optional
from pathlib import Path

In [3]:
class ToolManager:
    def __init__(self):
        self.tools = {
            "fs_read": self._fs_read,
            "fs_write": self._fs_write,
            "execute_bash": self._execute_bash,
            "introspect": self._introspect,
            "todo": self._todo
        }
        self._task_plans = {}

    def get_tools(self, requested_tools: List[str]) -> List[Dict]:
        """Convert tools to function calling format"""
        available_tools = []
        
        if "fs_read" in requested_tools:
            available_tools.append({
                "type": "function",
                "function": {
                    "name": "fs_read",
                    "description": "Read files, directories, or search for patterns. Use mode='Line' to read file contents, mode='Directory' to list files, mode='Search' to find patterns.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "operations": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "mode": {
                                            "type": "string", 
                                            "enum": ["Line", "Directory", "Search"],
                                            "description": "Must be exactly 'Line' (read file), 'Directory' (list files), or 'Search' (find pattern)"
                                        },
                                        "path": {"type": "string"},
                                        "pattern": {"type": "string"}
                                    },
                                    "required": ["mode", "path"]
                                }
                            }
                        },
                        "required": ["operations"]
                    }
                }
            })


        if "fs_write" in requested_tools:
            available_tools.append({
                "type": "function",
                "function": {
                    "name": "fs_write",
                    "description": "Create and edit files",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "command": {"type": "string", "enum": ["create", "str_replace", "append"]},
                            "path": {"type": "string"},
                            "file_text": {"type": "string"},
                            "old_str": {"type": "string"},
                            "new_str": {"type": "string"}
                        },
                        "required": ["command", "path"]
                    }
                }
            })

        if "execute_bash" in requested_tools:
            available_tools.append({
                "type": "function",
                "function": {
                    "name": "execute_bash",
                    "description": "Execute bash commands",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "command": {"type": "string"}
                        },
                        "required": ["command"]
                    }
                }
            })


        
        if "introspect" in requested_tools:
            available_tools.append({
                "type": "function",
                "function": {
                    "name": "introspect",
                    "description": "Get CLI capabilities",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {"type": "string"}
                        },
                        "required": ["query"]
                    }
                }
            })
        
        if "todo" in requested_tools:
            available_tools.append({
                "type": "function",
                "function": {
                    "name": "todo",
                    "description": "Break down tasks into steps",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "task": {"type": "string"},
                            "action": {"type": "string", "enum": ["plan", "execute", "status"]}
                        },
                        "required": ["task", "action"]
                    }
                }
            })
        
        return available_tools